In [103]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement
from os import chdir, getcwd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import csv
import pickle 
#from dfply import *

import os 
from os import chdir, getcwd
import datetime
import shapefile as shp
import pandas as pd
import geopandas as gpd
from shapely import geometry
import math
from os import listdir
from os.path import isfile, join
import glob

In [119]:
os.chdir('/home/jupyter-wndlovu/') # change working directory

wd=getcwd()
#wd

In [120]:
# add county files
# 
def fileInput(filepath):
    """This function is used to read input data (.csv files) stored in folders"""
    
    path =  filepath# landouse folder path
    files = [f for f in listdir(path) if isfile(join(path, f))] # read files from folder
    dfs_list = []  # List to store the dataframes

    for file in files: # read files and save them a list of dataframes
        file_path = os.path.join(path, file)
        df = pd.read_csv(file_path) 
        dfs_list.append(df)
        
    return(dfs_list)


def fileList(dataframe):
    """This function is used to group the gridmet, soils and canopy cover dataframes by the crop, irrigation management and county"""
    
    group_dataframe = dataframe.groupby('crop_mn_codeyear')

# list to store the dfs
    county_list = []

# create separate dfs
    for i, j in group_dataframe:
        county_list.append(j.copy())
        
    return(county_list)


# Input data

In [121]:
gridmet_list = fileInput(wd + '/eggs/gmd4_gridMET')
lai_list = fileInput(wd + '/eggs/leaf_area_index') 
soils_list = fileInput(wd + '/eggs/gmd4_soils_county')

# GridMET wrangling

In [122]:
# clean the datasets by the 
gridmet_df = pd.concat(gridmet_list, ignore_index = True)
                         
                    
gridmet_df = gridmet_df.assign(tmmn = gridmet_df.tmmn-273.15, # convert to celcius
                    tmmx = gridmet_df.tmmx-273.15,
                    date_ymd = pd.to_datetime(gridmet_df['date_ymd'], format='%Y%m%d'))

gridmet_df['Year'] = gridmet_df['date_ymd'].dt.year

# rename variables
gridmet_df  = gridmet_df.rename(columns = {
                                'tmmn':'MinTemp',
                                'tmmx':'MaxTemp',
                                'pr':'Precipitation',
                                'eto':'ReferenceET',
                                'date_ymd':'Date'
                                })

gridmet_df = gridmet_df[['crop_mn_codeyear',
                         'MinTemp', 
                         'MaxTemp',
                         'Precipitation', 
                         'ReferenceET',
                         'Date',
                         'Year'
                        ]]

gridmet_df = gridmet_df.sort_values(by = 'Date')
#gridmet_df

In [123]:
gridmet_county = fileList(gridmet_df)
len(gridmet_county)

68

In [124]:
#gridmet_county[0]

# Leaf Area Index Wrangling

In [136]:
lai_df = pd.concat(lai_list, ignore_index = True)

lai_df = lai_df.assign(cc = ((100.5*(1-np.exp(-0.6*(lai_df['Lai_500m'])))**1.2))/100, # for corn
                      date = pd.to_datetime(lai_df['date_ymd'], format='%Y%m%d')) # calc canopy cover Hsiao et al. (2009)

lai_df['Year'] = lai_df['date'].dt.year

lai_df = lai_df[['crop_mn_codeyear', 'Year', 'date', 'Lai_500m', 'cc']]

lai_df = lai_df.sort_values(by = 'date')

In [137]:
lai_df.sample(n=60)

,crop_mn_codeyear,Year,date,Lai_500m,cc
44704,8_Sheridan,2010,2010-08-21,10.925786,1.003285
45033,7_Gove,2010,2010-09-30,4.898893,0.941544
2464,4_Sheridan,2018,2018-10-16,3.958658,0.893916
44570,8_Wallace,2010,2010-08-05,11.694047,1.003918
2072,8_Decatur,2018,2018-08-29,12.214546,1.004208
45647,7_Sheridan,2010,2010-12-11,2.609828,0.758653
3491,6_Wallace,2017,2017-02-10,2.268722,0.704388
12122,1_Decatur,2011,2011-12-11,2.350889,0.718428
29064,2_Wallace,2008,2008-07-03,6.385076,0.978903
19288,6_Gove,2014,2014-04-23,5.069758,0.947697


In [127]:
lai_county = fileList(lai_df) 

# add an index variable
lai_county = [df.assign(index=range(len(df))) for df in lai_county]
lai_county = [df.assign(County=df['crop_mn_codeyear'].str.split('_').str[1]) for df in lai_county]
#lai_county

# Soils wrangling

In [128]:
soils_df = pd.concat(soils_list)

soils_df = soils_df[['crop_mn_codeyear', 'Year', 'system:index', 'mean']]

In [129]:
soils_df['variable'] = soils_df['system:index'].str.replace(r'^.*?(?=[a-z])', '', regex=True) # remove all numbers before the firct chr
soils_df['soil_var'] = soils_df['variable'].str[:-21] # drop the last 21 characters
soils_df['var'] = soils_df['soil_var'].str.rsplit('_', n=2).str[0] # get the soil param
soils_df['depth'] =soils_df['soil_var'].str.extract(r'(\d+_\d+)') # soil depth
soils_df = soils_df[['crop_mn_codeyear', 'Year', 'depth', 'var', 'mean']]


In [130]:
# pivot table to match aquacrop input format
soils_df_pivot = soils_df.pivot_table(index=['crop_mn_codeyear', 'Year', 'depth'],
                                 columns="var", values="mean")

soils_df_pivot = soils_df_pivot.assign(om = (10**(soils_df_pivot['om'])), # unit conversion
                     ksat = (10**(soils_df_pivot['ksat'])))                                         

soils_df_pivot = soils_df_pivot.reset_index()
#soils_df_pivot

In [131]:
soils_county = fileList(soils_df_pivot) 
len(soils_county)

68

# Create dictionary with dataframes for each county, mngt and crop combination

In [132]:
county_comb = gridmet_df['crop_mn_codeyear'].unique() # get all unique ids
#county_comb

In [133]:
input_dict = {}
comb_len = len(county_comb)

# loop through lists to form dict
for i in range(comb_len):
    key = str(gridmet_county[i]['crop_mn_codeyear'].unique())
    #print(key)
    value = (gridmet_county[i], lai_county[i], soils_county[i])
    input_dict[key] = value

In [134]:
#print(input_dict.keys())

In [135]:
# save dict as pickle
with open(wd + '/eggs/data/input_dict.pickle', 'wb') as input_data: # county crop managemnt
    pickle.dump(input_dict, input_data) 

# Next Step - Calibration